In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as mpl
import pymc as pm
import arviz as az
import pandas as pd
import os
from datetime import datetime
import sys
from math import atan,cos,sin
from vd_class import vd_8dof
from point import Point

### Set up the vehilce model for the acceleration test

In [ ]:
# A zero steering function
def zero_st(t):
    return 0 *t



# Used for obtaining the state from the chrono vehicle
n1  = 100
n2 = 850


# The time duration of the simulation
st_time = 1.0
end_time = 8.5


# The times for which we want the simulation
time  = np.arange(st_time,end_time,0.01)
t_eval= np.arange(st_time,end_time,0.01)


max_th = 0.5

# Now we construct the throttle function
throt1 = Point(0.5,0)
throt2 = Point(4.5,max_th)
ramp_throt1 = throt1.get_eq(throt2)
throt3 = Point(4.5,max_th)
throt4 = Point(8.5,0)
ramp_throt2 = throt3.get_eq(throt4)

def ramp_throt(t):
    if(t<4.5):
        return ramp_throt1(t)
    else:
        return ramp_throt2(t)

    
def brake_tor(t):
    return 0
state = pd.read_csv("data/acc.csv",sep=',',header='infer')
## For the data with Cxf = 10e5
# state = pd.read_csv("calib_mod_simp_acc_slow.csv",sep=',',header='infer')

st = {'x' : state['x'][n1],'y':state['y'][n1],'u':state['vx'][n1],'v':state['vy'][n1],'psi':state['yaw'][n1],
'phi':state['roll'][n1],'wx':state['roll_rate'][n1],'wz':state['yaw_rate'][n1],
'wlf' : state['wlf'][n1],'wlr' : state['wlr'][n1],'wrf' : state['wrf'][n1],'wrr' : state['wrr'][n1]}

vehicle = vd_8dof(states = st)

# Set the steering and the throttle functions we just created above
vehicle.set_steering(zero_st)
vehicle.set_throttle(ramp_throt,gr=0.3*0.2)
vehicle.set_braking(brake_tor)
vehicle.debug = 0

### Get the data and chain files

In [ ]:
# The chain file
filename = "20220612_024952"
idata = az.from_netcdf('./results/' + filename + ".nc")
dataFileName = "data/vd_chrono_acc_3.npy"
##The data in npy format
with open(dataFileName, 'rb') as f:
    data = np.load(f)


In [ ]:
save = False

### Set up the posteriors and the priors on parameters of interest

In [ ]:
shape = idata['posterior']['Cxf'].shape

posterior_Cxf = np.asarray(idata['posterior']['Cxf']).reshape(shape[0]*shape[1],)
posterior_Cxr = np.asarray(idata['posterior']['Cxr']).reshape(shape[0]*shape[1],)


prior_Cxf = np.random.uniform(low=1000,high=50000,size=(shape[0]*shape[1]),)
prior_Cxr = np.random.uniform(low=1000,high=50000,size=(shape[0]*shape[1]),)


### Sample from the posterior

In [ ]:
#Number of posterior lines
no_post = 100

#Random points to sample
random_points = np.random.randint(low=0,high=shape[0]*shape[1],size=(no_post,),dtype=int)

#The posterior data lines - Longitudinal Velocity, LF angular velocity and RF angular velocity
posterior_LOV = np.empty((no_post,time.shape[0]))
posterior_LFW = np.empty((no_post,time.shape[0]))
posterior_LRW = np.empty((no_post,time.shape[0]))


for i in range(0,no_post):
    vehicle.update_params(m=2097.85,muf=127.866,mur=129.98,a= 1.6889,b =1.6889,h = 0.713,cf = 1.82,cr = 1.82,
                          Jx = 1289,Jz = 4519,Jxz = 3.265,Cf=50000,Cr=50000,r0=0.47,ktf=326332,ktr=326332,
                          krof=37500,kror=37500,brof=15000,bror=15000,hrcf=0.379,hrcr=0.327,Jw=11,
                          Cxf =posterior_Cxf[random_points[i]],Cxr =posterior_Cxr[random_points[i]],rr=0.0175)
    
    mod = vehicle.solve_half_impl(t_span = [t_eval[0],t_eval[-1]],t_eval = t_eval,tbar = 5e-3)
    mod = np.transpose(mod)
    vehicle.reset_state(init_state=st)
    posterior_LOV[i] = mod[2,:]
    posterior_LFW[i] = mod[8,:]
    posterior_LRW[i] = mod[9,:]

### Sample from the prior

In [ ]:
#Number of prior lines
no_post = 100

#Random points to sample
random_points = np.random.randint(low=0,high=shape[0]*shape[1],size=(no_post,),dtype=int)

#The prior data lines - Longitudinal Velocity, LF angular velocity and RF angular velocity
prior_LOV = np.empty((no_post,time.shape[0]))
prior_LFW = np.empty((no_post,time.shape[0]))
prior_LRW = np.empty((no_post,time.shape[0]))


for i in range(0,no_post):
    vehicle.update_params(m=2097.85,muf=127.866,mur=129.98,a= 1.6889,b =1.6889,h = 0.713,cf = 1.82,cr = 1.82,
                          Jx = 1289,Jz = 4519,Jxz = 3.265,Cf=50000,Cr=50000,r0=0.47,ktf=326332,ktr=326332,
                          krof=37500,kror=37500,brof=15000,bror=15000,hrcf=0.379,hrcr=0.327,Jw=11,
                          Cxf =prior_Cxf[random_points[i]],Cxr =prior_Cxr[random_points[i]],rr=0.0175)
    
    mod = vehicle.solve_half_impl(t_span = [t_eval[0],t_eval[-1]],t_eval = t_eval,tbar = 5e-3)
    mod = np.transpose(mod)
    vehicle.reset_state(init_state=st)
    prior_LOV[i] = mod[2,:]
    prior_LFW[i] = mod[8,:]
    prior_LRW[i] = mod[9,:]

### Posterior and prior expectation

In [ ]:
#Posterior
post_exp_LOV = posterior_LOV.mean(axis=0)
post_exp_LFW = posterior_LFW.mean(axis=0)
post_exp_LRW = posterior_LRW.mean(axis=0)
#prior
prior_exp_LOV = prior_LOV.mean(axis=0)
prior_exp_LFW = prior_LFW.mean(axis=0)
prior_exp_LRW = prior_LRW.mean(axis=0)

### Plotting the mean posterior

In [ ]:
## To plot the mean posterior 
post_Cxf = idata['posterior']['Cxf'].mean()
post_Cxr = idata['posterior']['Cxr'].mean()

vehicle.update_params(m=2097.85,muf=127.866,mur=129.98,a= 1.6889,b =1.6889,h = 0.713,cf = 1.82,cr = 1.82,
                      Jx = 1289,Jz = 4519,Jxz = 3.265,Cf=50000,Cr=50000,r0=0.47,ktf=326332,ktr=326332,
                          krof=37500,kror=37500,brof=15000,bror=15000,hrcf=0.379,hrcr=0.327,Jw=11,
                      Cxf =post_Cxf,Cxr =post_Cxr,rr=0.0175)
mod = vehicle.solve_half_impl(t_span = [t_eval[0],t_eval[-1]],t_eval = t_eval,tbar = 5e-3)
mod = np.transpose(mod)
vehicle.reset_state(init_state=st)


post_mean_LOV = mod[2,:].reshape(-1,1)
post_mean_LFW = mod[8,:].reshape(-1,1)
post_mean_LRW = mod[9,:].reshape(-1,1)

### Plotting the mean prior

In [ ]:
## To plot the mean priorerior 
prior_Cxf = prior_Cxf.mean()
prior_Cxr = prior_Cxr.mean()

vehicle.update_params(m=2097.85,muf=127.866,mur=129.98,a= 1.6889,b =1.6889,h = 0.713,cf = 1.82,cr = 1.82,
                      Jx = 1289,Jz = 4519,Jxz = 3.265,Cf=50000,Cr=50000,r0=0.47,ktf=326332,ktr=326332,
                          krof=37500,kror=37500,brof=15000,bror=15000,hrcf=0.379,hrcr=0.327,Jw=11,
                      Cxf =prior_Cxf,Cxr =prior_Cxr,rr=0.0175)
mod = vehicle.solve_half_impl(t_span = [t_eval[0],t_eval[-1]],t_eval = t_eval,tbar = 5e-3)
mod = np.transpose(mod)
vehicle.reset_state(init_state=st)


prior_mean_LOV = mod[2,:].reshape(-1,1)
prior_mean_LFW = mod[8,:].reshape(-1,1)
prior_mean_LRW = mod[9,:].reshape(-1,1)

## Prior plots

### Longitudinal Velocity

In [ ]:
mpl.figure(figsize = (6,6))

for i in range(0,no_post):
    mpl.plot(time,prior_LOV[i,:],'y',alpha = 0.2)

# mpl.plot(time,data[])
mpl.plot(time,data[0,:],'r',label ='noisy data')
mpl.plot(time,prior_exp_LOV,'k--',linewidth=2,label='prior expectation')
mpl.plot(time,prior_mean_LOV,'g--',linewidth=2,label='prior mean')
mpl.title("Longitudinal Velocity - Prior",fontsize = 20)
mpl.xlabel('Time (S)',fontsize = 12)
mpl.ylabel('Longitudinal Velocity (m/s)',fontsize = 12)
mpl.legend()
if(save):
    mpl.savefig('./images/LOV_acc_prior_1.png',facecolor = 'w')

### LF Angular velocity

In [ ]:
mpl.figure(figsize = (6,6))

for i in range(0,no_post):
    mpl.plot(time,prior_LFW[i,:],'y',alpha = 0.2)

mpl.plot(time,data[1,:],'r',label ='noisy data')
mpl.plot(time,prior_exp_LFW,'k--',linewidth=2,label='prior expectation')
mpl.plot(time,prior_mean_LFW,'g--',linewidth=2,label='prior mean')
mpl.title("Angular Velocity (LF) - Prior",fontsize = 20)
mpl.xlabel('Time (S)',fontsize = 12)
mpl.ylabel('Angular Velocity (rps)',fontsize = 12)
mpl.legend()
if(save):
    mpl.savefig('./images/LFW_acc_prior_1.png',facecolor = 'w')

### LR Angular velocity

In [ ]:
mpl.figure(figsize = (6,6))

for i in range(0,no_post):
    mpl.plot(time,prior_LRW[i,:],'y',alpha = 0.2)

mpl.plot(time,data[2,:],'r',label ='noisy data')
mpl.plot(time,prior_exp_LRW,'k--',linewidth=2,label='prior expectation')
mpl.plot(time,prior_mean_LRW,'g--',linewidth=2,label='prior mean')
mpl.title("Angular Velocity (LR) - Prior",fontsize = 20)
mpl.xlabel('Time (S)',fontsize = 12)
mpl.ylabel('Angular Velocity (rps)',fontsize = 12)
mpl.legend()
if(save):
    mpl.savefig('./images/LRW_acc_prior_1.png',facecolor = 'w')

## Posterior plots

### Longitudinal Velocity

In [ ]:
mpl.figure(figsize = (6,6))

for i in range(0,no_post):
    mpl.plot(time,posterior_LOV[i,:],'b',alpha=0.2)

mpl.plot(time,data[0,:],'r',label ='noisy data')
mpl.plot(time,post_exp_LOV,'k',linewidth = 2,label='posterior expectiation')
mpl.plot(time,post_mean_LOV,'g',linewidth=2,label='posterior mean')
mpl.title("Longitudinal Velocity - Posterior",fontsize = 20)
mpl.xlabel('Time (S)',fontsize = 12)
mpl.ylabel('Longitudinal Velocity (m/s)',fontsize = 12)
mpl.legend()
if(save):
    mpl.savefig('./images/LOV_acc_post_1.png',facecolor = 'w')

### Angular velocity (LF)

In [ ]:
mpl.figure(figsize = (6,6))

for i in range(0,no_post):
    mpl.plot(time,posterior_LFW[i,:],'b',alpha=0.2)

mpl.plot(time,data[1,:],'r',label ='noisy data')
mpl.plot(time,post_exp_LFW,'k',linewidth = 2,label='posterior expectiation')
mpl.plot(time,post_mean_LFW,'g',linewidth=2,label='posterior mean')
mpl.title("Angular Velocity (LF) - Posterior",fontsize = 20)
mpl.xlabel('Time (S)',fontsize = 12)
mpl.ylabel('Angular Velocity (rps)',fontsize = 12)
mpl.legend()
if(save):
    mpl.savefig('./images/LFW_acc_post_1.png',facecolor = 'w')

### Angular velocity (LR)

In [ ]:
mpl.figure(figsize = (6,6))

for i in range(0,no_post):
    mpl.plot(time,posterior_LRW[i,:],'b',alpha=0.2)

mpl.plot(time,data[2,:],'r',label ='noisy data')
mpl.plot(time,post_exp_LRW,'k',linewidth = 2,label='posterior expectiation')
mpl.plot(time,post_mean_LRW,'g',linewidth=2,label='posterior mean')
mpl.title("Angular Velocity (LR) - Posterior",fontsize = 20)
mpl.xlabel('Time (S)',fontsize = 12)
mpl.ylabel('Angular Velocity (rps)',fontsize = 12)
mpl.legend()
if(save):
    mpl.savefig('./images/LRW_acc_post_1.png',facecolor = 'w')

### RMSE

### Longitudinal velocity

In [ ]:
msae_post_LOV = np.mean(np.sqrt((np.sum((posterior_LOV - data[0,:])**2,axis=1)/data.shape[1])))

In [ ]:
msae_post_LOV

In [ ]:
msae_prior_LOV = np.mean(np.sqrt((np.sum((prior_LOV - data[0,:])**2,axis=1)/data.shape[1])))

In [ ]:
msae_prior_LOV

### LF 

In [ ]:
msae_post_LFW = np.mean(np.sqrt((np.sum((posterior_LFW - data[1,:])**2,axis=1)/data.shape[1])))
msae_post_LFW

In [ ]:
msae_prior_LFW = np.mean(np.sqrt((np.sum((prior_LFW - data[1,:])**2,axis=1)/data.shape[1])))
msae_prior_LFW

### LR

In [ ]:
msae_post_LRW = np.mean(np.sqrt((np.sum((posterior_LRW - data[2,:])**2,axis=1)/data.shape[1])))
msae_post_LRW

In [ ]:
msae_prior_LRW = np.mean(np.sqrt((np.sum((prior_LRW - data[2,:])**2,axis=1)/data.shape[1])))
msae_prior_LRW